# SageMaker Deployment for Solar Power Generation ML Pipeline

This notebook demonstrates how to deploy the trained solar power generation model to Amazon SageMaker for production inference.

## Overview
- Deploy trained model to SageMaker endpoint
- Test real-time inference
- Set up batch transform jobs
- Configure auto-scaling and monitoring

## 1. Setup and Configuration

In [ ]:
import boto3
import sagemaker
from sagemaker import get_execution_role
from sagemaker.sklearn.estimator import SKLearn
from sagemaker.sklearn.model import SKLearnModel
import pandas as pd
import numpy as np
import json
import joblib
from datetime import datetime
import os

# SageMaker session and role
sagemaker_session = sagemaker.Session()
role = get_execution_role()
region = boto3.Session().region_name

print(f"SageMaker role: {role}")
print(f"AWS region: {region}")

In [ ]:
# Configuration
bucket = sagemaker_session.default_bucket()
prefix = 'solar-power-ml'
model_name = 'solar-power-neural-network'
endpoint_name = f'solar-power-endpoint-{datetime.now().strftime("%Y%m%d-%H%M%S")}'

print(f"S3 bucket: {bucket}")
print(f"Model prefix: {prefix}")
print(f"Endpoint name: {endpoint_name}")

## 2. Prepare Model Artifacts

In [ ]:
# Load the trained model and preprocessor
import sys
sys.path.append('../src')

# Load model artifacts
model_path = '../models/trained/neural_network_model.pkl'
scaler_path = '../models/trained/scaler.pkl'

if os.path.exists(model_path) and os.path.exists(scaler_path):
    model = joblib.load(model_path)
    scaler = joblib.load(scaler_path)
    print("✅ Model and scaler loaded successfully")
else:
    print("❌ Model files not found. Please run the training notebook first.")

In [ ]:
# Create model artifacts directory
os.makedirs('../sagemaker_deployment/model_artifacts', exist_ok=True)

# Copy model files to deployment directory
import shutil
shutil.copy(model_path, '../sagemaker_deployment/model_artifacts/')
shutil.copy(scaler_path, '../sagemaker_deployment/model_artifacts/')

print("Model artifacts prepared for deployment")

## 3. Create SageMaker Training Script

In [ ]:
%%writefile ../sagemaker_deployment/code/train.py
"""
SageMaker Training Script for Solar Power Generation Model
"""

import argparse
import os
import pandas as pd
import numpy as np
import joblib
from sklearn.neural_network import MLPRegressor
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error
import json

def model_fn(model_dir):
    """Load model for inference"""
    model = joblib.load(os.path.join(model_dir, 'model.pkl'))
    scaler = joblib.load(os.path.join(model_dir, 'scaler.pkl'))
    return {'model': model, 'scaler': scaler}

def input_fn(request_body, request_content_type):
    """Parse input data"""
    if request_content_type == 'application/json':
        input_data = json.loads(request_body)
        return pd.DataFrame(input_data)
    elif request_content_type == 'text/csv':
        return pd.read_csv(request_body)
    else:
        raise ValueError(f"Unsupported content type: {request_content_type}")

def predict_fn(input_data, model_dict):
    """Make predictions"""
    model = model_dict['model']
    scaler = model_dict['scaler']
    
    # Scale input data
    scaled_data = scaler.transform(input_data)
    
    # Make predictions
    predictions = model.predict(scaled_data)
    
    return predictions

def output_fn(prediction, content_type):
    """Format output"""
    if content_type == 'application/json':
        return json.dumps(prediction.tolist())
    elif content_type == 'text/csv':
        return ','.join(map(str, prediction))
    else:
        raise ValueError(f"Unsupported content type: {content_type}")

if __name__ == '__main__':
    parser = argparse.ArgumentParser()
    
    # Hyperparameters
    parser.add_argument('--hidden_layer_sizes', type=str, default='100,50,25')
    parser.add_argument('--alpha', type=float, default=0.001)
    parser.add_argument('--max_iter', type=int, default=300)
    parser.add_argument('--random_state', type=int, default=42)
    
    # SageMaker specific arguments
    parser.add_argument('--model-dir', type=str, default=os.environ.get('SM_MODEL_DIR'))
    parser.add_argument('--train', type=str, default=os.environ.get('SM_CHANNEL_TRAIN'))
    
    args = parser.parse_args()
    
    # Load training data
    train_data = pd.read_csv(os.path.join(args.train, 'train.csv'))
    
    # Prepare features and target
    feature_columns = [col for col in train_data.columns if col != 'generation']
    X = train_data[feature_columns]
    y = train_data['generation']
    
    # Split data
    X_train, X_test, y_train, y_test = train_test_split(
        X, y, test_size=0.2, random_state=args.random_state
    )
    
    # Scale features
    scaler = StandardScaler()
    X_train_scaled = scaler.fit_transform(X_train)
    X_test_scaled = scaler.transform(X_test)
    
    # Parse hidden layer sizes
    hidden_layers = tuple(map(int, args.hidden_layer_sizes.split(',')))
    
    # Train model
    model = MLPRegressor(
        hidden_layer_sizes=hidden_layers,
        alpha=args.alpha,
        max_iter=args.max_iter,
        random_state=args.random_state,
        early_stopping=True,
        validation_fraction=0.2
    )
    
    model.fit(X_train_scaled, y_train)
    
    # Evaluate model
    y_pred = model.predict(X_test_scaled)
    rmse = np.sqrt(mean_squared_error(y_test, y_pred))
    r2 = r2_score(y_test, y_pred)
    mae = mean_absolute_error(y_test, y_pred)
    
    print(f"Model Performance:")
    print(f"RMSE: {rmse:.4f}")
    print(f"R² Score: {r2:.4f}")
    print(f"MAE: {mae:.4f}")
    
    # Save model and scaler
    joblib.dump(model, os.path.join(args.model_dir, 'model.pkl'))
    joblib.dump(scaler, os.path.join(args.model_dir, 'scaler.pkl'))
    
    # Save feature names
    with open(os.path.join(args.model_dir, 'feature_names.json'), 'w') as f:
        json.dump(feature_columns, f)
    
    # Save model metrics
    metrics = {
        'rmse': rmse,
        'r2_score': r2,
        'mae': mae,
        'n_features': len(feature_columns),
        'n_samples': len(train_data)
    }
    
    with open(os.path.join(args.model_dir, 'metrics.json'), 'w') as f:
        json.dump(metrics, f)
    
    print("Model training completed successfully!")

## 4. Create SageMaker Inference Script

In [ ]:
%%writefile ../sagemaker_deployment/code/inference.py
"""
SageMaker Inference Script for Solar Power Generation Model
"""

import os
import json
import joblib
import pandas as pd
import numpy as np
from io import StringIO

def model_fn(model_dir):
    """
    Load the model and preprocessor for inference
    """
    try:
        model = joblib.load(os.path.join(model_dir, 'model.pkl'))
        scaler = joblib.load(os.path.join(model_dir, 'scaler.pkl'))
        
        # Load feature names if available
        feature_names_path = os.path.join(model_dir, 'feature_names.json')
        if os.path.exists(feature_names_path):
            with open(feature_names_path, 'r') as f:
                feature_names = json.load(f)
        else:
            feature_names = None
        
        return {
            'model': model,
            'scaler': scaler,
            'feature_names': feature_names
        }
    except Exception as e:
        raise Exception(f"Error loading model: {str(e)}")

def input_fn(request_body, request_content_type):
    """
    Parse and preprocess the input data
    """
    try:
        if request_content_type == 'application/json':
            input_data = json.loads(request_body)
            
            # Handle different JSON formats
            if isinstance(input_data, dict):
                if 'instances' in input_data:
                    # SageMaker batch format
                    df = pd.DataFrame(input_data['instances'])
                else:
                    # Single instance format
                    df = pd.DataFrame([input_data])
            elif isinstance(input_data, list):
                # List of instances
                df = pd.DataFrame(input_data)
            else:
                raise ValueError("Invalid JSON format")
                
        elif request_content_type == 'text/csv':
            # CSV format
            df = pd.read_csv(StringIO(request_body))
            
        else:
            raise ValueError(f"Unsupported content type: {request_content_type}")
        
        return df
        
    except Exception as e:
        raise Exception(f"Error parsing input: {str(e)}")

def predict_fn(input_data, model_dict):
    """
    Make predictions using the loaded model
    """
    try:
        model = model_dict['model']
        scaler = model_dict['scaler']
        feature_names = model_dict.get('feature_names')
        
        # Ensure correct feature order if feature names are available
        if feature_names:
            # Check if all required features are present
            missing_features = set(feature_names) - set(input_data.columns)
            if missing_features:
                raise ValueError(f"Missing features: {missing_features}")
            
            # Reorder columns to match training data
            input_data = input_data[feature_names]
        
        # Scale the input data
        scaled_data = scaler.transform(input_data)
        
        # Make predictions
        predictions = model.predict(scaled_data)
        
        # Ensure predictions are non-negative (solar generation can't be negative)
        predictions = np.maximum(predictions, 0)
        
        return predictions
        
    except Exception as e:
        raise Exception(f"Error making predictions: {str(e)}")

def output_fn(predictions, content_type):
    """
    Format the predictions for output
    """
    try:
        if content_type == 'application/json':
            # Return predictions with metadata
            output = {
                'predictions': predictions.tolist(),
                'model_name': 'solar_power_neural_network',
                'prediction_count': len(predictions),
                'units': 'kWh'
            }
            return json.dumps(output)
            
        elif content_type == 'text/csv':
            # Return as CSV
            output_df = pd.DataFrame({
                'predicted_generation_kwh': predictions
            })
            return output_df.to_csv(index=False)
            
        else:
            # Default: return as plain text
            return ','.join(map(str, predictions))
            
    except Exception as e:
        raise Exception(f"Error formatting output: {str(e)}")

## 5. Upload Training Data to S3

In [ ]:
# Upload processed training data to S3
train_data_path = '../data/processed/processed_solar_data.csv'

if os.path.exists(train_data_path):
    # Upload to S3
    train_s3_path = sagemaker_session.upload_data(
        path=train_data_path,
        bucket=bucket,
        key_prefix=f'{prefix}/data/train'
    )
    print(f"Training data uploaded to: {train_s3_path}")
else:
    print("❌ Training data not found. Please run preprocessing first.")

## 6. Create and Train SageMaker Estimator

In [ ]:
# Create SageMaker estimator
sklearn_estimator = SKLearn(
    entry_point='train.py',
    source_dir='../sagemaker_deployment/code',
    role=role,
    instance_type='ml.m5.large',
    framework_version='0.23-1',
    py_version='py3',
    hyperparameters={
        'hidden_layer_sizes': '100,50,25',
        'alpha': 0.001,
        'max_iter': 300,
        'random_state': 42
    },
    output_path=f's3://{bucket}/{prefix}/model-artifacts',
    sagemaker_session=sagemaker_session
)

print("SageMaker estimator created successfully")

In [ ]:
# Start training job
print("Starting SageMaker training job...")
sklearn_estimator.fit({'train': train_s3_path})
print("Training job completed!")

## 7. Deploy Model to Endpoint

In [ ]:
# Deploy model to endpoint
print(f"Deploying model to endpoint: {endpoint_name}")

predictor = sklearn_estimator.deploy(
    initial_instance_count=1,
    instance_type='ml.t2.medium',
    endpoint_name=endpoint_name
)

print(f"✅ Model deployed successfully to endpoint: {endpoint_name}")

## 8. Test Real-time Inference

In [ ]:
# Load test data for inference
test_data_path = '../data/processed/processed_solar_data.csv'

if os.path.exists(test_data_path):
    test_df = pd.read_csv(test_data_path)
    
    # Get feature columns (exclude target)
    feature_columns = [col for col in test_df.columns if col != 'generation']
    
    # Select a few samples for testing
    test_samples = test_df[feature_columns].head(5)
    actual_values = test_df['generation'].head(5)
    
    print("Test samples prepared:")
    print(test_samples)
else:
    print("❌ Test data not found")

In [ ]:
# Make predictions
try:
    # Convert to JSON format
    test_input = test_samples.to_dict('records')
    
    # Make prediction
    predictions = predictor.predict(test_input)
    
    # Parse predictions
    if isinstance(predictions, str):
        pred_data = json.loads(predictions)
        predicted_values = pred_data['predictions']
    else:
        predicted_values = predictions
    
    # Display results
    results_df = pd.DataFrame({
        'Actual': actual_values.values,
        'Predicted': predicted_values,
        'Difference': actual_values.values - predicted_values
    })
    
    print("\n🎯 Prediction Results:")
    print(results_df)
    
    # Calculate metrics
    rmse = np.sqrt(np.mean((actual_values.values - predicted_values) ** 2))
    mae = np.mean(np.abs(actual_values.values - predicted_values))
    
    print(f"\n📊 Test Metrics:")
    print(f"RMSE: {rmse:.4f} kWh")
    print(f"MAE: {mae:.4f} kWh")
    
except Exception as e:
    print(f"❌ Error making predictions: {e}")

## 9. Set up Batch Transform

In [ ]:
# Create batch transform job for large-scale predictions
from sagemaker.transformer import Transformer

# Create transformer
transformer = Transformer(
    model_name=sklearn_estimator.model_name,
    instance_count=1,
    instance_type='ml.m5.large',
    output_path=f's3://{bucket}/{prefix}/batch-predictions',
    sagemaker_session=sagemaker_session
)

print("Batch transformer created successfully")

In [ ]:
# Prepare batch data (optional - for demonstration)
batch_data_path = '../data/processed/batch_test_data.csv'

# Create a sample batch file
if os.path.exists(test_data_path):
    batch_df = test_df[feature_columns].head(100)  # First 100 samples
    batch_df.to_csv(batch_data_path, index=False)
    
    # Upload batch data to S3
    batch_s3_path = sagemaker_session.upload_data(
        path=batch_data_path,
        bucket=bucket,
        key_prefix=f'{prefix}/data/batch'
    )
    
    print(f"Batch data uploaded to: {batch_s3_path}")
    
    # Run batch transform (uncomment to execute)
    # transformer.transform(
    #     data=batch_s3_path,
    #     content_type='text/csv',
    #     split_type='Line'
    # )
    # print("Batch transform job started")
else:
    print("Test data not available for batch processing")

## 10. Monitor Endpoint Performance

In [ ]:
# Set up CloudWatch monitoring
import boto3
from datetime import datetime, timedelta

cloudwatch = boto3.client('cloudwatch')

def get_endpoint_metrics(endpoint_name, start_time, end_time):
    """Get endpoint metrics from CloudWatch"""
    
    metrics = {
        'Invocations': [],
        'ModelLatency': [],
        'OverheadLatency': []
    }
    
    for metric_name in metrics.keys():
        try:
            response = cloudwatch.get_metric_statistics(
                Namespace='AWS/SageMaker',
                MetricName=metric_name,
                Dimensions=[
                    {
                        'Name': 'EndpointName',
                        'Value': endpoint_name
                    },
                ],
                StartTime=start_time,
                EndTime=end_time,
                Period=300,  # 5 minutes
                Statistics=['Average', 'Sum']
            )
            metrics[metric_name] = response['Datapoints']
        except Exception as e:
            print(f"Error getting {metric_name}: {e}")
    
    return metrics

# Get metrics for the last hour
end_time = datetime.utcnow()
start_time = end_time - timedelta(hours=1)

metrics = get_endpoint_metrics(endpoint_name, start_time, end_time)
print(f"📊 Endpoint metrics for {endpoint_name}:")
for metric_name, datapoints in metrics.items():
    if datapoints:
        latest = datapoints[-1]
        print(f"{metric_name}: {latest.get('Average', 'N/A')}")
    else:
        print(f"{metric_name}: No data available")

## 11. Auto-scaling Configuration

In [ ]:
# Configure auto-scaling for the endpoint
autoscaling_client = boto3.client('application-autoscaling')

# Register scalable target
resource_id = f'endpoint/{endpoint_name}/variant/AllTraffic'

try:
    autoscaling_client.register_scalable_target(
        ServiceNamespace='sagemaker',
        ResourceId=resource_id,
        ScalableDimension='sagemaker:variant:DesiredInstanceCount',
        MinCapacity=1,
        MaxCapacity=5
    )
    
    # Create scaling policy
    autoscaling_client.put_scaling_policy(
        PolicyName=f'{endpoint_name}-scaling-policy',
        ServiceNamespace='sagemaker',
        ResourceId=resource_id,
        ScalableDimension='sagemaker:variant:DesiredInstanceCount',
        PolicyType='TargetTrackingScaling',
        TargetTrackingScalingPolicyConfiguration={
            'TargetValue': 70.0,
            'PredefinedMetricSpecification': {
                'PredefinedMetricType': 'SageMakerVariantInvocationsPerInstance'
            },
            'ScaleOutCooldown': 300,
            'ScaleInCooldown': 300
        }
    )
    
    print("✅ Auto-scaling configured successfully")
    print("- Min instances: 1")
    print("- Max instances: 5")
    print("- Target: 70 invocations per instance")
    
except Exception as e:
    print(f"⚠️ Auto-scaling setup failed: {e}")
    print("This is normal if auto-scaling is already configured")

## 12. Cleanup (Optional)

In [ ]:
# Uncomment to delete the endpoint (to avoid charges)
# predictor.delete_endpoint()
# print(f"Endpoint {endpoint_name} deleted")

print("\n⚠️ Remember to delete the endpoint when no longer needed to avoid charges:")
print(f"predictor.delete_endpoint()")
print(f"\nOr use AWS CLI:")
print(f"aws sagemaker delete-endpoint --endpoint-name {endpoint_name}")

## Summary

This notebook demonstrated:

1. **Model Deployment**: Successfully deployed the trained neural network model to SageMaker
2. **Real-time Inference**: Tested the endpoint with sample data
3. **Batch Processing**: Set up batch transform for large-scale predictions
4. **Monitoring**: Configured CloudWatch monitoring for endpoint performance
5. **Auto-scaling**: Set up automatic scaling based on traffic

### Key Features:
- **Production Ready**: Scalable endpoint with monitoring
- **Cost Optimized**: Auto-scaling to handle variable load
- **Robust**: Error handling and validation
- **Flexible**: Supports both JSON and CSV input formats

### Next Steps:
1. Set up automated retraining pipeline
2. Implement A/B testing for model updates
3. Add custom metrics and alarms
4. Integrate with application APIs